In [1]:
import sys
sys.path.append('..')

In [4]:
# Import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from src.db import ProductTable, SellerTable, engine, SessionLocal

In [5]:
db_session = SessionLocal()

### Getting data from .parquet files

In [4]:
output_folder = '../src/parser/data8'

data = []
for file in os.listdir(output_folder):
    if file.endswith('.parquet'):
        df = pd.read_parquet(f'{output_folder}/{file}')
        data.append(df)
data = pd.concat(data)

In [ ]:
data.head()

,title,price,characteristics,description,views,date,location,link,seller_id,today_views,...,rating,reviews,subscribers,subscriptions,registered,done_deals,active_deals,docs_confirmed,phone_confirmed,response_time
0,"iPhone 16 Pro Max, 256 ГБ",124390,"{'Встроенная память': '256 ГБ', 'Модель': 'iPh...",В нaличии Арple iPhonе 16 Рrо Мaх 256 GB Dеsеr...,4668,· 11 ноября в 15:30,"Санкт-Петербург, Кирочная ул., 5",https://www.avito.ru/sankt-peterburg/telefony/...,/brands/ibox,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"iPhone 12 mini, 256 ГБ",19500,"{'IMEI': None, 'Встроенная память': '256 ГБ', ...",«Офис Джобса» — федеральная сеть cпециaлизирoв...,6833,· 18 декабря в 09:43,"Санкт-Петербург, ул. Ефимова, 4А",https://www.avito.ru/sankt-peterburg/telefony/...,/brands/ofisjobsa,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"iPhone 13, 128 ГБ",29590,"{'IMEI': None, 'Встроенная память': '128 ГБ', ...",Tелeфoн iРhоne 13Цвет: РозовыйПамять: 128GbKом...,6113,· вчера в 21:51,"Санкт-Петербург, пл. Стачек, 9",https://www.avito.ru/sankt-peterburg/telefony/...,/brands/cc43b5570cd46264d357070bb2cb1f69,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"iPhone 12, 64 ГБ",18490,"{'IMEI': None, 'Встроенная память': '64 ГБ', '...",🏆Удoбcтво и дoступность с МIRАРНONЕ - пoлучите...,1071,· 5 декабря в 10:12,"Санкт-Петербург, Садовая ул., 38",https://www.avito.ru/sankt-peterburg/telefony/...,/brands/i59503469,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"iPhone 14 Pro Max, 256 ГБ",69999,"{'IMEI': '**********57985', 'Встроенная память...",Комплект полный как было с магазина + стекло и...,377,· 18 декабря в 19:55,"Санкт-Петербург, ул. Владимира Пчелинцева, 4",https://www.avito.ru/sankt-peterburg/telefony/...,efa418db-cce4-4625-8b61-14597638d25b,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Getting data from database using sqlAlchemy

In [7]:
products = db_session.query(ProductTable).all()
sellers = db_session.query(SellerTable).all()

In [26]:
# products to dataframe
products_data = []
for product in products:
    products_data.append(product.__dict__)
products_data = pd.DataFrame(products_data)

# deserialize the characteristics and about from json
products_data['characteristics'] = products_data['characteristics'].apply(lambda x: eval(x) if isinstance(x, str) else x)
products_data['about'] = products_data['about'].apply(lambda x: eval(x) if isinstance(x, str) else x)
products_data.drop(columns=['_sa_instance_state'], inplace=True)

# sellers to dataframe
sellers_data = []
for seller in sellers:
    sellers_data.append(seller.__dict__)
sellers_data = pd.DataFrame(sellers_data)
sellers_data.drop(columns=['_sa_instance_state'], inplace=True)


                       title  \
0          iPhone 15, 128 ГБ   
1  iPhone 16 Pro Max, 256 ГБ   
2          iPhone 14, 128 ГБ   
3          iPhone 16, 256 ГБ   
4          iPhone 13, 128 ГБ   

                                     characteristics  views  \
0  {'Состояние': 'Новое', 'Производитель': 'Apple...   4113   
1  {'Состояние': 'Новое', 'Производитель': 'Apple...   4668   
2  {'Состояние': 'Новое', 'Производитель': 'Apple...  17339   
3  {'Состояние': 'Новое', 'Производитель': 'Apple...   1141   
4  {'Состояние': 'Новое', 'Производитель': 'Apple...   2177   

                                       location  today_views  \
0         Санкт-Петербург, 2-я Советская ул., 7            0   
1              Санкт-Петербург, Кирочная ул., 5            2   
2         Санкт-Петербург, 2-я Советская ул., 7            0   
3         Санкт-Петербург, 2-я Советская ул., 7            0   
4  Санкт-Петербург, Большая Московская ул., 2/1            0   

                                         de

In [28]:
# merge the data
df = products_data
df = df.merge(sellers_data, on='seller_id')

df.head()

,title,characteristics,views,location,today_views,description,link,price,date,seller_id,...,reviews,subscriptions,done_deals,docs_confirmed,response_time,subscribers,rating,registered,active_deals,phone_confirmed
0,"iPhone 15, 128 ГБ","{'Состояние': 'Новое', 'Производитель': 'Apple...",4113,"Санкт-Петербург, 2-я Советская ул., 7",0,"iPhone 15 128GB, чёрный",https://www.avito.ru/sankt-peterburg/telefony/...,71490.0,· 13 декабря в 23:09,/brands/i2137433,...,764.0,927.0,2372.0,False,Отвечает за несколько часов,5.0,4.9,На Авито с февраля 2011,680.0,True
1,"iPhone 16 Pro Max, 256 ГБ","{'Состояние': 'Новое', 'Производитель': 'Apple...",4668,"Санкт-Петербург, Кирочная ул., 5",2,В нaличии Арple iPhonе 16 Рrо Мaх 256 GB Dеsеr...,https://www.avito.ru/sankt-peterburg/telefony/...,124390.0,· 11 ноября в 15:30,/brands/ibox,...,997.0,480.0,6.0,True,Отвечает около 30 минут,2.0,5.0,На Авито с октября 2012,156.0,True
2,"iPhone 14, 128 ГБ","{'Состояние': 'Новое', 'Производитель': 'Apple...",17339,"Санкт-Петербург, 2-я Советская ул., 7",0,iPhone 14 128 GВ (Nаnо Sim + еSim) cиний,https://www.avito.ru/sankt-peterburg/telefony/...,60890.0,· 1 декабря в 12:57,/brands/i2137433,...,764.0,927.0,2372.0,False,Отвечает за несколько часов,5.0,4.9,На Авито с февраля 2011,680.0,True
3,"iPhone 16, 256 ГБ","{'Состояние': 'Новое', 'Производитель': 'Apple...",1141,"Санкт-Петербург, 2-я Советская ул., 7",0,"iPhоne 16 256GB, рoзoвый",https://www.avito.ru/sankt-peterburg/telefony/...,106990.0,· 20 декабря в 17:27,/brands/i2137433,...,764.0,927.0,2372.0,False,Отвечает за несколько часов,5.0,4.9,На Авито с февраля 2011,680.0,True
4,"iPhone 13, 128 ГБ","{'Состояние': 'Новое', 'Производитель': 'Apple...",2177,"Санкт-Петербург, Большая Московская ул., 2/1",0,Аррle iРhonе 13 на гарантии с зaводcкими пломб...,https://www.avito.ru/sankt-peterburg/telefony/...,44990.0,· 10 ноября в 19:05,/brands/tehnomag,...,145.0,0.0,0.0,True,Отвечает за несколько часов,425.0,4.9,На Авито с ноября 2023,276.0,True


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            60 non-null     object 
 1   characteristics  60 non-null     object 
 2   views            60 non-null     int64  
 3   location         60 non-null     object 
 4   today_views      60 non-null     int64  
 5   description      60 non-null     object 
 6   link             60 non-null     object 
 7   price            60 non-null     float64
 8   date             60 non-null     object 
 9   seller_id        60 non-null     object 
 10  about            46 non-null     object 
 11  name             60 non-null     object 
 12  reviews          56 non-null     float64
 13  subscriptions    57 non-null     float64
 14  done_deals       57 non-null     float64
 15  docs_confirmed   60 non-null     bool   
 16  response_time    54 non-null     object 
 17  subscribers      5